In [ ]:
!pip install --upgrade subgrounds

## Python Subground

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from collections import defaultdict
from dotenv import load_dotenv
import os
from subgrounds import Subgrounds

### Draft

In [216]:
# Initialize Subgraph
load_dotenv()
api_key = os.getenv('THEGRAPH_API')
sg = Subgrounds()
v3_pool = sg.load_subgraph(f"https://gateway-arbitrum.network.thegraph.com/api/{api_key}/subgraphs/id/2kmbjLfKbEDz2bKxrg7FtcnBg84ghBHkGrLXENbDBgEv")

In [228]:
current_time = dt.datetime.now()
previous_hour_time = current_time - dt.timedelta(hours=1)
previous_hour_time

datetime.datetime(2024, 5, 15, 12, 54, 54, 48366)

In [233]:
dt.datetime.fromtimestamp(first_block_timestamp)

datetime.datetime(2024, 5, 15, 14, 9, 5)

In [248]:
def get_block_range_previous_hour():
    current_time = dt.datetime.now()
    previous_hour_time = current_time - dt.timedelta(hours=1)
    
    # Get the first block of the previous hour
    first_block_query = v3_pool.Query.supplies(
        first=1,
        orderBy=v3_pool.Supply.blockTimestamp,
        orderDirection='asc',
        where={
            'blockTimestamp_gte': int(previous_hour_time.timestamp())
        }
    )
    first_block_df = sg.query_df([first_block_query.blockNumber, first_block_query.blockTimestamp])
    
    if first_block_df.empty:
        raise ValueError("No blocks found for the start of the previous hour.")
    
    first_block_number = first_block_df['supplies_blockNumber'][0]
    first_block_timestamp = first_block_df['supplies_blockTimestamp'][0]
    
    # Get the last block of the previous hour
    last_block_query = v3_pool.Query.supplies(
        first=1,
        orderBy=v3_pool.Supply.blockTimestamp,
        orderDirection='desc',
        where={
            'blockTimestamp_lte': int(current_time.timestamp())
        }
    )
    last_block_df = sg.query_df([last_block_query.blockNumber, last_block_query.blockTimestamp])
    
    if last_block_df.empty:
        raise ValueError("No blocks found for the end of the previous hour.")
    
    last_block_number = last_block_df['supplies_blockNumber'][0]
    last_block_timestamp = last_block_df['supplies_blockTimestamp'][0]
    
    return first_block_number, last_block_number, first_block_timestamp, last_block_timestamp

first_block_number, last_block_number, first_block_timestamp, last_block_timestamp = get_block_range_previous_hour()

In [250]:

print(f"Running pipeline to get the events between the blocks {last_block_number} and {first_block_number} at {dt.datetime.now()} ")


# The relation token decimals -- Address is hardcoded but we can improve later for scalability purposes :)
token_decimals = {
    '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'.lower(): ['USDC',6],  # USDC
    '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'.lower(): ['USDT',6],  # USDT
    '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(): ['ETH',18], # WETH
}

# Querying Deposits on AAVE V3 Arbitrum with pagination

# Construct the base query for supplies
supplies = v3_pool.Query.supplies(
    orderBy=v3_pool.Supply.blockNumber,
    orderDirection='asc',
    where={
        'blockNumber_gte': int(first_block_number),
        'blockNumber_lte': int(last_block_number),
        'reserve_in': list(token_decimals.keys())
    }
)

# Fetch all supplies data
df_supplies = sg.query_df([supplies])
df_supplies.head()

Running pipeline to get the events between the blocks 211582851 and 211569105 at 2024-05-15 14:41:13.594473 


,supplies_id,supplies_reserve,supplies_user,supplies_onBehalfOf,supplies_amount,supplies_referralCode,supplies_blockNumber,supplies_blockTimestamp,supplies_transactionHash
0,0x55ee6b7a1bef79e726c63d4bea433d31ccb86b9ae86b...,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,0xecd4bd3121f9fd604ffac631bf6d41ec12f1fafb,0x7dfe079fba8ba9abef26c07ea7eaa139dc8b9d4e,6493093270000000,0,211569655,1715790567,0x55ee6b7a1bef79e726c63d4bea433d31ccb86b9ae86b...
1,0x15d5ffb2aef6331426126d1717ccb2414f0fce5b64ad...,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,0xecd4bd3121f9fd604ffac631bf6d41ec12f1fafb,0xaa07364675bd65f8cd40e7477a3ae59a9a1c6048,16620619082784813004,0,211570230,1715790713,0x15d5ffb2aef6331426126d1717ccb2414f0fce5b64ad...
2,0xc2bfa25f7d6573384776dfa7a1fbcff66963f9d81248...,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,0x28201c152dc5b69a86fa54fcfd21bca4c0eff3ba,0x91a6c53faa15ab1546d83c63e4552f34e4dd98d2,291916217500379,0,211570554,1715790796,0xc2bfa25f7d6573384776dfa7a1fbcff66963f9d81248...
3,0x2a4050267116b4a76c605de76ec11e1c47f93d774c3f...,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,0xecd4bd3121f9fd604ffac631bf6d41ec12f1fafb,0x2268e60d8d31b43550366f76511650876d8c3e08,655000000000000000,0,211570979,1715790904,0x2a4050267116b4a76c605de76ec11e1c47f93d774c3f...
4,0x9e0ae3e2cddeb4a07ddb5180c0fd6ddb7bb92310a0ab...,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,0xc47bb288178ea40bf520a91826a3dee9e0dbfa4c,0xc47bb288178ea40bf520a91826a3dee9e0dbfa4c,33243114656564368,0,211571379,1715791007,0x9e0ae3e2cddeb4a07ddb5180c0fd6ddb7bb92310a0ab...


## Offical Script

In [2]:
# Initialize Subgraph
load_dotenv()
api_key = os.getenv('THEGRAPH_API')
sg = Subgrounds()
v3_pool = sg.load_subgraph(f"https://gateway-arbitrum.network.thegraph.com/api/{api_key}/subgraphs/id/2kmbjLfKbEDz2bKxrg7FtcnBg84ghBHkGrLXENbDBgEv")

# Getting the first block and the last block of the previous hour
def get_block_range_previous_hour():
    current_time = dt.datetime.now()
    previous_hour_time = current_time - dt.timedelta(hours=1)
    
    # Get the first block of the previous hour
    first_block_query = v3_pool.Query.supplies(
        first=1,
        orderBy=v3_pool.Supply.blockTimestamp,
        orderDirection='asc',
        where={
            'blockTimestamp_gte': int(previous_hour_time.timestamp())
        }
    )
    first_block_df = sg.query_df([first_block_query.blockNumber, first_block_query.blockTimestamp])
    
    if first_block_df.empty:
        raise ValueError("No blocks found for the start of the previous hour.")
    
    first_block_number = first_block_df['supplies_blockNumber'][0]
    first_block_timestamp = first_block_df['supplies_blockTimestamp'][0]
    
    # Get the last block of the previous hour
    last_block_query = v3_pool.Query.supplies(
        first=1,
        orderBy=v3_pool.Supply.blockTimestamp,
        orderDirection='desc',
        where={
            'blockTimestamp_lte': int(current_time.timestamp())
        }
    )
    last_block_df = sg.query_df([last_block_query.blockNumber, last_block_query.blockTimestamp])
    
    if last_block_df.empty:
        raise ValueError("No blocks found for the end of the previous hour.")
    
    last_block_number = last_block_df['supplies_blockNumber'][0]
    last_block_timestamp = last_block_df['supplies_blockTimestamp'][0]
    
    return int(first_block_number), int(last_block_number), int(first_block_timestamp), int(last_block_timestamp)


# The relation token decimals -- Address is hardcoded but we can improve later for scalability purposes :)
token_decimals = {
    '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'.lower(): ['USDC',6],  # USDC
    '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'.lower(): ['USDT',6],  # USDT
    '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(): ['ETH',18], # WETH
}

# Adjust amount based on token decimals
def adjust_amount(token_address, amount):
    decimals = token_decimals[token_address][1]
    return amount / (10 ** decimals)


# Function to query and process events for a specific block range
def process_block_range(start_block, last_block):
    supplies_query = v3_pool.Query.supplies(
        where={
            'blockNumber_gte': start_block,
            'blockNumber_lte': last_block,
            'reserve_in': list(token_decimals.keys())
        }
    )
    withdraws_query = v3_pool.Query.withdraws(
        where={
            'blockNumber_gte': start_block,
            'reserve_in': list(token_decimals.keys())
        }
    )
    borrows_query = v3_pool.Query.borrows(
        where={
            'blockNumber_gte': start_block,
            'blockNumber_lte': last_block,
            'reserve_in': list(token_decimals.keys())
        }
    )
    repays_query = v3_pool.Query.repays(
        where={
            'blockNumber_gte': start_block,
            'blockNumber_lte': last_block,
            'reserve_in': list(token_decimals.keys())
        }
    )
    liquidationCalls_query = v3_pool.Query.liquidationCalls(
        where={
            'blockNumber_gte': start_block,
                'blockNumber_lte': last_block,
            'collateralAsset_in': list(token_decimals.keys())
        }
    )

    # Fetch data
    supplies_df = sg.query_df([supplies_query.id, supplies_query.reserve, supplies_query.user, supplies_query.amount, supplies_query.blockNumber, supplies_query.blockTimestamp])
    withdraws_df = sg.query_df([withdraws_query.id, withdraws_query.reserve, withdraws_query.user, withdraws_query.amount, withdraws_query.blockNumber, withdraws_query.blockTimestamp])
    borrows_df = sg.query_df([borrows_query.id, borrows_query.reserve, borrows_query.user, borrows_query.amount, borrows_query.blockNumber, borrows_query.blockTimestamp])
    repays_df = sg.query_df([repays_query.id, repays_query.reserve, repays_query.user, repays_query.amount, repays_query.blockNumber, repays_query.blockTimestamp])
    liquidationCalls_df = sg.query_df([liquidationCalls_query.id, liquidationCalls_query.collateralAsset, liquidationCalls_query.debtAsset, liquidationCalls_query.user, liquidationCalls_query.debtToCover, liquidationCalls_query.liquidatedCollateralAmount, liquidationCalls_query.blockNumber, liquidationCalls_query.blockTimestamp])

    # Process data
    user_balances = {}

    # Process supplies
    for _, row in supplies_df.iterrows():
        user = row['supplies_user']
        token = row['supplies_reserve']
        adjusted_amount = adjust_amount(token, row['supplies_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] += adjusted_amount

    # Process withdraws
    for _, row in withdraws_df.iterrows():
        user = row['withdraws_user']
        token = row['withdraws_reserve']
        adjusted_amount = adjust_amount(token, row['withdraws_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] -= adjusted_amount

    # Process borrows
    for _, row in borrows_df.iterrows():
        user = row['borrows_user']
        token = row['borrows_reserve']
        adjusted_amount = adjust_amount(token, row['borrows_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] -= adjusted_amount

    # Process repays
    for _, row in repays_df.iterrows():
        user = row['repays_user']
        token = row['repays_reserve']
        adjusted_amount = adjust_amount(token, row['repays_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] += adjusted_amount

    # Process liquidation calls
    for _, row in liquidationCalls_df.iterrows():
        user = row['liquidationCalls_user']
        token = row['liquidationCalls_collateralAsset']
        adjusted_amount = adjust_amount(token, row['liquidationCalls_liquidatedCollateralAmount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] -= adjusted_amount
    return user_balances

# Convert balances to DataFrame
def balances_to_dataframe(user_balances, block_number, block_timestamp):
    records = []
    for user, tokens in user_balances.items():
        for token, amount in tokens.items():
            records.append({
                'block_number': block_number,
                'timestamp': dt.datetime.fromtimestamp(block_timestamp),
                'owner_address': user,
                'token_address': token,
                'token_symbol': token_decimals[token][0],
                'token_amount': float(amount)
            })
    return pd.DataFrame(records)



if __name__ == "__main__":
    # Get the last block of the previous hour
    start_block, last_block, first_block_timestamp, last_block_timestamp = get_block_range_previous_hour() # Assign variables
    
    # Process the events for this block
    user_balances = process_block_range(start_block,last_block)
    
    # Convert to DataFrame
    df_balances = balances_to_dataframe(user_balances, last_block, last_block_timestamp)
    
    print(df_balances)

    # Save the final DataFrame to a CSV file
    df_balances.to_csv('net_supplied_amount_previous_hour.csv', index=False)

    block_number           timestamp  \
0      211598775 2024-05-15 15:37:42   
1      211598775 2024-05-15 15:37:42   
2      211598775 2024-05-15 15:37:42   
3      211598775 2024-05-15 15:37:42   
4      211598775 2024-05-15 15:37:42   
..           ...                 ...   
71     211598775 2024-05-15 15:37:42   
72     211598775 2024-05-15 15:37:42   
73     211598775 2024-05-15 15:37:42   
74     211598775 2024-05-15 15:37:42   
75     211598775 2024-05-15 15:37:42   

                                 owner_address  \
0   0xecd4bd3121f9fd604ffac631bf6d41ec12f1fafb   
1   0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251   
2   0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251   
3   0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251   
4   0x4f1bae6f00c32e8602821819614ca71a07411de8   
..                                         ...   
71  0xd68d601b4a8a0e782f7a2a36f93c7aada89f17aa   
72  0xf1f4aee1c202602daf066fbc8d8424928ec9e845   
73  0x66a9927f465c21a576b3ce5de00239323875e1d0   
74  0x0384af3a95757

In [3]:
df_balances

,block_number,timestamp,owner_address,token_address,token_symbol,token_amount
0,211598775,2024-05-15 15:37:42,0xecd4bd3121f9fd604ffac631bf6d41ec12f1fafb,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,8.875481
1,211598775,2024-05-15 15:37:42,0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,2.178603
2,211598775,2024-05-15 15:37:42,0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,USDC,-13911.433964
3,211598775,2024-05-15 15:37:42,0xf3c3f14dd7bdb7e03e6ebc3bc5ffc6d66de12251,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,USDT,-315253.405078
4,211598775,2024-05-15 15:37:42,0x4f1bae6f00c32e8602821819614ca71a07411de8,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,125.650000
...,...,...,...,...,...,...
71,211598775,2024-05-15 15:37:42,0xd68d601b4a8a0e782f7a2a36f93c7aada89f17aa,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,USDT,99.920568
72,211598775,2024-05-15 15:37:42,0xf1f4aee1c202602daf066fbc8d8424928ec9e845,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,USDT,93.624500
73,211598775,2024-05-15 15:37:42,0x66a9927f465c21a576b3ce5de00239323875e1d0,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,0.059875
74,211598775,2024-05-15 15:37:42,0x0384af3a95757b13e6499c6df0df5763742bf08a,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,USDT,17773.000000


In [80]:
withdraws_df.columns

Index(['withdraws_id', 'withdraws_reserve', 'withdraws_user',
       'withdraws_amount', 'withdraws_blockNumber',
       'withdraws_blockTimestamp'],
      dtype='object')

In [88]:
# The relation token decimals -- Address is hardcoded but we can improve later for scalability purposes :)
token_decimals = {
    '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'.lower(): ['USDC',6],  # USDC
    '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'.lower(): ['USDT',6],  # USDT
    '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(): ['ETH',18], # WETH
}

def adjust_amount(token_address, amount):
    decimals = token_decimals[token_address][1]
    return amount/ 10**decimals


# Data Prepatation
def process_data():
    user_balances = {}

    # Process supplies
    for _, row in supplies_df.iterrows():
        user = row['supplies_user']
        token = row['supplies_reserve']
        adjusted_amount = adjust_amount(token, row['supplies_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] += adjusted_amount

    # Process withdraws
    for _, row in withdraws_df.iterrows():
        user = row['withdraws_user']
        token = row['withdraws_reserve']
        adjusted_amount = adjust_amount(token, row['withdraws_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] -= adjusted_amount

    # Process borrows
    for _, row in borrows_df.iterrows():
        user = row['borrows_user']
        token = row['borrows_reserve']
        adjusted_amount = adjust_amount(token, row['borrows_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] -= adjusted_amount

    # Process repays
    for _, row in repays_df.iterrows():
        user = row['repays_user']
        token = row['repays_reserve']
        adjusted_amount = adjust_amount(token, row['repays_amount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] += adjusted_amount

    # Process liquidation calls
    for _, row in liquidationCalls_df.iterrows():
        user = row['liquidationCalls_user']
        token = row['liquidationCalls_collateralAsset']
        adjusted_amount = adjust_amount(token, row['liquidationCalls_liquidatedCollateralAmount'])

        if user not in user_balances:
            user_balances[user] = {}
        if token not in user_balances[user]:
            user_balances[user][token] = 0

        user_balances[user][token] += adjusted_amount
    return user_balances

user_balances = process_data()

# Convert to DataFrame and export in csv 
def balances_to_df(user_balances):
    records = []
    for user, tokens in user_balances.items():
        for token, amount in tokens.items():
            records.append({
                'block_number': last_block,
                'timestamp': time,
                'owner_address': user,
                'token_address': token,
                'token_symbol': token_decimals[token][0],
                'token_amount': float(amount)    
            })
    return pd.DataFrame(records)
df_balance = balances_to_df(user_balances)
df_balance.to_csv("aave3_arbitrum_netsupply.csv", index = False)




In [109]:
df_balance

,block_number,timestamp,owner_address,token_address,token_symbol,token_amount
0,211418496,2024-05-15 02:44:16,0xc09e69e79106861df5d289da88349f10e2dc6b5c,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,-74.804430
1,211418496,2024-05-15 02:44:16,0xe96d9171e0e4ae2b676a41361bb65e74a69ed724,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,0.020000
2,211418496,2024-05-15 02:44:16,0x13e965baafda80c7501c23cbed282d5e53e3066b,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,USDC,200.000250
3,211418496,2024-05-15 02:44:16,0x13e965baafda80c7501c23cbed282d5e53e3066b,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,USDT,-0.027271
4,211418496,2024-05-15 02:44:16,0x413c89b5b16c4e3b6182efd24c484a689e6f65c3,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,USDC,-1.206549
...,...,...,...,...,...,...
178,211418496,2024-05-15 02:44:16,0x089df687c476de42415e11988b16ae7a24d67010,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,0.043196
179,211418496,2024-05-15 02:44:16,0xc349a7d9166c2b4048d1823b5543d940168304e2,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,0.021587
180,211418496,2024-05-15 02:44:16,0x7849dcc0e82c3daf67619994aab85391fca75b71,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,0.151061
181,211418496,2024-05-15 02:44:16,0xb8af4834fcf05fb594bca20a1ed940ca8772ea05,0x82af49447d8a07e3bd95bd0d56f35241523fbab1,ETH,1.552325


### API Integration

In [33]:
load_dotenv()
api_key = os.getenv('THEGRAPH_API')
graphql_end_point= f"https://gateway-arbitrum.network.thegraph.com/api/{api_key}/subgraphs/id/2kmbjLfKbEDz2bKxrg7FtcnBg84ghBHkGrLXENbDBgEv"
query = """
   {
      supplies: supplies(first: 1000, where: {reserve_in: ["0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8", "0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9", "0x82aF49447D8a07e3bd95BD0d56f35241523fBab1"]}, orderBy: blockNumber, orderDirection: asc) {
        id
        reserve
        user
        amount
        blockNumber
        blockTimestamp
      }
}
"""
response = requests.post(graphql_end_point, json={'query': query})
response.json()['data']


{'supplies': [{'amount': '10000000000000000',
   'blockNumber': '7998442',
   'blockTimestamp': '1647446445',
   'id': '0xe4be4e6fc7c105f66b2361853a5083d7f5aedc189332f822f5d766fdfc263fb907000000',
   'reserve': '0x82af49447d8a07e3bd95bd0d56f35241523fbab1',
   'user': '0xc09e69e79106861df5d289da88349f10e2dc6b5c'},
  {'amount': '950000000000000000',
   'blockNumber': '7998790',
   'blockTimestamp': '1647447073',
   'id': '0x2af7e7adf185d7e9212925c8ae44b1fec1e7bd9342a0df3ce2934de501fad23b07000000',
   'reserve': '0x82af49447d8a07e3bd95bd0d56f35241523fbab1',
   'user': '0xc09e69e79106861df5d289da88349f10e2dc6b5c'},
  {'amount': '200000000000000000',
   'blockNumber': '7998872',
   'blockTimestamp': '1647447170',
   'id': '0x555f0359241bdb2af36dc157ae00d61ecda9519e920767d389600fc241f12b8d06000000',
   'reserve': '0x82af49447d8a07e3bd95bd0d56f35241523fbab1',
   'user': '0xc09e69e79106861df5d289da88349f10e2dc6b5c'},
  {'amount': '1000000000000000000',
   'blockNumber': '8000775',
   'blockTim